<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/Training/3_Training_3_DPO_from_SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Direct Preference Optimization for Critical Question Generation
In this notebook we train the pretrained vanilla LLM with DPO Training.
- **Model:** [ricostaedeli/Meta-Llama-3.1-8B-Instruct_SFT_2](https://huggingface.co/ricostaedeli/Meta-Llama-3.1-8B-Instruct_SFT_2)
- **Dataset:** [processed_train_data_filtered_dpo.json](../Data/Processed/processed_train_data_filtered_dpo.json)
- **Frameworks:** Unsloth, HuggingFace, transformers, bitsandbytes

## Setup
First we define some constant values and also install all needed libraries



### Installation

In [ ]:
!pip install --no-deps xformers triton unsloth_zoo
!pip install sentencepiece protobuf huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import shutil
import os
import torch
from datasets import load_dataset
from trl import DPOTrainer
import logging
from transformers import TrainingArguments, EarlyStoppingCallback, IntervalStrategy, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

### Colab
This part is only relevant when using the notebook in google colab

In [ ]:
from google.colab import userdata, drive

In [ ]:
drive.mount('/content/drive')
token = userdata.get('GITHUB')

Mounted at /content/drive


Clone GitHub Repository to directly push generated files

In [ ]:
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 1394, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 1394 (delta 184), reused 126 (delta 92), pack-reused 1136 (from 2)
Receiving objects: 100% (1394/1394), 49.82 MiB | 25.29 MiB/s, done.
Resolving deltas: 100% (786/786), done.
Updating files: 100% (116/116), done.


### Path Variables and Logger

In [ ]:
################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

TRAINING_NUMBER = 3
BASE_MODEL_REPO = "ricostaedeli/Meta-Llama-3.1-8B-Instruct_SFT"
MODEL_NAME = "Meta-Llama-3.1-8B-Instruct_SFT_DPO"

################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

train_dataset_path = "/content/NLP2025_CQG/Data/Processed/processed_train_data_filtered_dpo.json"

log_base_path = f"/content/NLP2025_CQG/Training/Logs/Traing_{TRAINING_NUMBER}/Tensorboard/"
os.makedirs(log_base_path, exist_ok=True)

log_file_path = f"/content/NLP2025_CQG/Logs/training_{TRAINING_NUMBER}.log"

model_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_finetuned/"
os.makedirs(model_save_path, exist_ok=True)

model_lora_adapter_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_lora_adapters/"
os.makedirs(model_lora_adapter_save_path, exist_ok=True)


checkpoint_dir = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)


################################################################################
#######################   LOGGER                ################################
################################################################################

# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_file_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

In [ ]:
logger.info("--------  Start with Training  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Model: {MODEL_NAME}')
logger.info(f'Training number: {TRAINING_NUMBER}')

INFO:__main__:--------  Start with Training  -------------
INFO:__main__:Device selected: cuda
INFO:__main__:Model: Meta-Llama-3.1-8B-Instruct_SFT_2_DPO_1
INFO:__main__:Training number: 7


## Training Parameters

In [ ]:
################################################################################
#######################   Unlsoth Parameters    ################################
################################################################################

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


################################################################################
#######################   PEFT Parameters       ################################
################################################################################

r = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]
lora_alpha = 16
lora_dropout = 0 # Supports any, but = 0 is optimized
bias = "none"    # Supports any, but = "none" is optimized
use_gradient_checkpointing = "unsloth" # True or "unsloth" for very long context
random_state = 3407
use_rslora = False  # Unsloth supports rank stabilized LoRA
loftq_config = None # And LoftQ

### Unsloth

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_REPO,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = r,
    target_modules = target_modules,
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    bias = bias,
    use_gradient_checkpointing = use_gradient_checkpointing,
    random_state = random_state,
    use_rslora = use_rslora,
    loftq_config = loftq_config,
)

Unsloth 2025.5.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Load and preprocess dataset
The raw dataset [SocratiQ](https://github.com/NUS-IDS/eacl23_soqg/tree/main) has a label at the begining of the context. We have to remove that and also tokenize the input for the model training.

In [ ]:
dataset = load_dataset('json', data_files=train_dataset_path)

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'schema', 'id', 'context'],
        num_rows: 3128
    })
})


## Training

## Prepare Tokenizer

In [ ]:
from unsloth.chat_templates import CHAT_TEMPLATES
print(list(CHAT_TEMPLATES.keys()))


['unsloth', 'zephyr', 'chatml', 'mistral', 'llama', 'vicuna', 'vicuna_old', 'vicuna old', 'alpaca', 'gemma', 'gemma_chatml', 'gemma2', 'gemma2_chatml', 'llama-3', 'llama3', 'phi-3', 'phi-35', 'phi-3.5', 'llama-3.1', 'llama-31', 'llama-3.2', 'llama-3.3', 'llama-32', 'llama-33', 'qwen-2.5', 'qwen-25', 'qwen25', 'qwen2.5', 'phi-4', 'gemma-3', 'gemma3', 'qwen-3', 'qwen3']


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

## Prepare Dataset

In [ ]:
print(dataset['train'][0])

{'prompt': [{'content': "Generate one critical question addressing the provided context. Ensure it matches the schema: FearAppeal\n\nContext: implication_consequences: The argument isn't that school teachers' compensation is adequate. The argument is that everyone should be paid a living wage. Someone asked how much a living wage was, and OP responded with an estimation of 40-50k.", 'role': 'user'}], 'chosen': [{'content': 'How much would a big Mac be if every employee made 50k a year?', 'role': 'assistant'}], 'rejected': [{'content': 'Is paying school teachers a living wage a way to ensure their well-being and job satisfaction?', 'role': 'assistant'}], 'schema': 'FearAppeal', 'id': 0, 'context': "implication_consequences: The argument isn't that school teachers' compensation is adequate. The argument is that everyone should be paid a living wage. Someone asked how much a living wage was, and OP responded with an estimation of 40-50k."}


In [ ]:
column_names = list(dataset["train"].features)
def apply_dpo_template(example):
  if all(k in example.keys() for k in ("chosen", "rejected","prompt")):

    # For DPO, the inputs are triples of (prompt, chosen, rejected), where `chosen` and `rejected` are the final turn of a dialogue
    prompt_messages = example["prompt"]
    chosen_messages = example["chosen"]
    rejected_messages = example["rejected"]

    example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
    example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
    example["text_prompt"] = tokenizer.apply_chat_template(prompt_messages, tokenize=False)
  return example

dataset = dataset.map(apply_dpo_template,remove_columns=column_names,
          desc="Formatting comparisons with prompt template",)

In [ ]:
print(dataset['train'][0])

{'text_chosen': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHow much would a big Mac be if every employee made 50k a year?<|eot_id|>', 'text_rejected': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIs paying school teachers a living wage a way to ensure their well-being and job satisfaction?<|eot_id|>', 'text_prompt': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGenerate one critical question addressing the provided context. Ensure it matches the schema: FearAppeal\n\nContext: implication_consequences: The argument isn't that school teachers' compensation is adequ

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text_chosen', 'text_rejected', 'text_prompt'],
        num_rows: 3128
    })
})


In [ ]:
dataset = dataset['train'].train_test_split(test_size=0.2, shuffle=True, seed=42)


In [ ]:
for split in ["train", "test"]:
    dataset[split] = dataset[split].rename_columns(
        {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
    )

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'prompt'],
        num_rows: 2502
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'prompt'],
        num_rows: 626
    })
})


## Define Training variables

In [ ]:
from trl import DPOConfig
training_args = DPOConfig(
        do_eval=True,
        eval_strategy = "steps",
        save_strategy = "steps",
        eval_steps = 5,
        logging_steps = 1,
        max_steps = 40,
        warmup_ratio = 0.1,
        per_device_train_batch_size = 20,
        gradient_accumulation_steps = 4,
        per_device_eval_batch_size = 1,
        learning_rate = 5e-6,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        save_total_limit = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = checkpoint_dir,
        report_to = "tensorboard",
        logging_dir = log_base_path
)


from unsloth import PatchDPOTrainer
PatchDPOTrainer()

trainer = DPOTrainer(
    model,
    ref_model=None,
    args=training_args,
    beta=0.1,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    max_length = 1024,
    max_prompt_length = 512
)


In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

logger.info(f"GPU  Information before Training")
logger.info(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
logger.info(f"{start_gpu_memory} GB of memory reserved.")

INFO:__main__:GPU  Information before Training
INFO:__main__:GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
INFO:__main__:7.625 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,502 | Num Epochs = 2 | Total steps = 40
O^O/ \_/ \    Batch size per device = 20 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (20 x 4 x 1) = 80
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
5,0.686300,0.691160,0.012830,0.008460,0.557508,0.004370,-184.449951,-167.941071,-0.521420,-0.539907,0,0,0,0
10,0.665000,0.687760,0.005556,-0.005688,0.610224,0.011244,-184.522690,-168.082550,-0.523199,-0.541632,No Log,No Log,No Log,No Log
15,0.637400,0.686333,0.008858,-0.005255,0.635783,0.014113,-184.489670,-168.078232,-0.519168,-0.537939,No Log,No Log,No Log,No Log
20,0.597600,0.683974,0.011710,-0.007240,0.666134,0.018950,-184.461151,-168.098083,-0.522574,-0.541235,No Log,No Log,No Log,No Log
25,0.589500,0.682351,0.013072,-0.009267,0.682109,0.022339,-184.447540,-168.118362,-0.519646,-0.538463,No Log,No Log,No Log,No Log
30,0.528000,0.677976,0.007736,-0.023678,0.757188,0.031413,-184.500900,-168.262451,-0.521158,-0.540285,No Log,No Log,No Log,No Log
35,0.519000,0.680307,0.003738,-0.023127,0.675719,0.026865,-184.540863,-168.256958,-0.522371,-0.541318,No Log,No Log,No Log,No Log
40,0.537500,0.678061,0.013334,-0.018016,0.723642,0.031350,-184.444901,-168.205841,-0.521086,-0.540132,No Log,No Log,No Log,No Log


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

logger.info(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
logger.info(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
logger.info(f"Peak reserved memory = {used_memory} GB.")
logger.info(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
logger.info(f"Peak reserved memory % of max memory = {used_percentage} %.")
logger.info(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Save
Save the finetuned model

### Save Lora delta weights

In [ ]:
model.save_pretrained(model_lora_adapter_save_path)  # Local saving
tokenizer.save_pretrained(model_lora_adapter_save_path)
logger.info(f"Saved LoRA adapters to {model_lora_adapter_save_path}")

INFO:__main__:Saved LoRA adapters to /content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_7/Model/Meta-Llama-3.1-8B-Instruct_SFT_2_DPO_1_lora_adapters/


In [ ]:
token = userdata.get('HF_TOKEN')

# Push Lora weights to HF
model.push_to_hub_merged(f"ricostaedeli/{MODEL_NAME}-lora", tokenizer, save_method = "lora", token = token)

### Saving merged model

Save merged model

In [ ]:
# Merge to 16bit and save local
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "merged_16bit",)
  logger.info(f"Saved merged model in 16bit to {model_save_path}")

# Merge to 16bit and push to HF
if True:
  token = userdata.get('HF_TOKEN')
  model.push_to_hub_merged(f"ricostaedeli/{MODEL_NAME}", tokenizer, save_method="merged_16bit", token=token, private=True)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="$log_dir"

In [ ]:
os.chdir("NLP2025_CQG")
!ls


In [ ]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabriga.ch"


commit_message = f"Training Number: {TRAINING_NUMBER}, Training logs in Google Drive"
!git add .
!git commit -m "{commit_message}"
!git push